# Simple Evolutionary Algorithm

## Import Libraries

In [1]:
import torch
import pretty_midi
from collections import Counter
from IPython.display import Audio, display

## Helper Functions

### Load MIDI to File & Convert to Sequence

In [2]:
def midi_to_sequence(midi_file):
    """Load a MIDI file and convert it into a sequence of note numbers."""
    midi_data = pretty_midi.PrettyMIDI(midi_file)
    notes = []
    for instrument in midi_data.instruments:
        for note in instrument.notes:
            notes.append(note.pitch)
    
    return torch.tensor(notes)

### Calculate Transition Probabilities

In [3]:
def calculate_transition_probabilities(midi_sequence):
    """Calculate transition probabilities from a MIDI sequence."""
    transitions = [(midi_sequence[i], midi_sequence[i + 1]) for i in range(len(midi_sequence) - 1)]
    transition_counts = Counter(transitions)
    total_transitions = sum(transition_counts.values())
    transition_probs = {k: v / total_transitions for k, v in transition_counts.items()}
    
    return transition_probs

### Evaluate Fitness Based on Transition Probabilities

In [4]:
def fitness_function(sequence, transition_probs):
    """Evaluate the fitness of a sequence based on transition probabilities."""
    fitness = 0
    
    for i in range(len(sequence) - 1):
        pair = (sequence[i].item(), sequence[i + 1].item())
        fitness += transition_probs.get(pair, 0)  # Reward common transitions
    
    return fitness

## Define Evolutionary Algorithm Functions

### Initialize Population from MIDI Data

In [5]:
def initialize_population_from_midi(midi_sequence, pop_size, seq_length):
    """Initialize a population of sequences from MIDI data."""
    population = []
    
    max_start = len(midi_sequence) - seq_length
    
    for _ in range(pop_size):
        start_idx = torch.randint(0, max_start, (1,)).item()
        sequence = midi_sequence[start_idx:start_idx + seq_length]
        population.append(sequence)
    
    return torch.stack(population)

### Select Parents Based on Fitness Scores

In [6]:
def select_parents(population, fitness_scores, num_parents):
    """Select the best sequences as parents."""
    _, indices = torch.topk(fitness_scores, num_parents)
    
    return population[indices]

### Crossover to Create Offspring

In [7]:
def crossover(parent1, parent2):
    """Combine two parents to create a child sequence."""
    point = torch.randint(1, len(parent1) - 1, (1,)).item()  # Random crossover point
    child = torch.cat((parent1[:point], parent2[point:]))
    
    return child

def create_offspring(parents, num_offspring):
    """Create offspring from selected parents."""
    offspring = []
    
    for _ in range(num_offspring):
        parent1, parent2 = parents[torch.randint(0, len(parents), (2,))]
        child = crossover(parent1, parent2)
        offspring.append(child)
    
    return torch.stack(offspring)

### Mutation

In [8]:
def mutate(sequence, mutation_rate=0.1):
    """Introduce random mutations into a sequence."""
    mutation_mask = torch.rand(sequence.size()) < mutation_rate
    random_notes = torch.randint(note_range[0], note_range[1], sequence.size())
    
    sequence[mutation_mask] = random_notes[mutation_mask]
    
    return sequence

def mutate_population(population, mutation_rate=0.1):
    """Mutate the entire population."""
    for i in range(len(population)):
        population[i] = mutate(population[i], mutation_rate)
    
    return population

## Run the Evolutionary Algorithm

In [9]:
# Main function to run the evolutionary algorithm
def evolutionary_algorithm_midi(num_generations, midi_sequence, seq_length, 
                                pop_size, num_parents, mutation_rate):
    """Run the evolutionary algorithm on MIDI data."""
    population = initialize_population_from_midi(midi_sequence, pop_size, seq_length)
    transition_probs = calculate_transition_probabilities(midi_sequence)
    
    for generation in range(num_generations):
        fitness_scores = torch.tensor([fitness_function(seq, transition_probs) for seq in population])
        parents = select_parents(population, fitness_scores, num_parents)
        offspring = create_offspring(parents, pop_size - num_parents)
        offspring = mutate_population(offspring, mutation_rate)
        
        # The new population is a combination of parents and offspring
        population = torch.cat((parents, offspring))
        
        best_fitness = fitness_scores.max().item()
        print(f"Generation {generation + 1}, Best Fitness: {best_fitness}")
    
    best_sequence = population[torch.argmax(fitness_scores)]
    
    return best_sequence

# Parameters
midi_file = 'midi_files/Moonlight-Sonata.mid'
midi_sequence = midi_to_sequence(midi_file)
note_range = (60, 72)  # Example range for MIDI notes

# Run the algorithm
best_sequence = evolutionary_algorithm_midi(num_generations=20, midi_sequence=midi_sequence, 
                                            seq_length=8, pop_size=10, num_parents=4, 
                                            mutation_rate=0.1)
print(f"Best sequence: {best_sequence}")

Generation 1, Best Fitness: 0
Generation 2, Best Fitness: 0
Generation 3, Best Fitness: 0
Generation 4, Best Fitness: 0
Generation 5, Best Fitness: 0
Generation 6, Best Fitness: 0
Generation 7, Best Fitness: 0
Generation 8, Best Fitness: 0
Generation 9, Best Fitness: 0
Generation 10, Best Fitness: 0
Generation 11, Best Fitness: 0
Generation 12, Best Fitness: 0
Generation 13, Best Fitness: 0
Generation 14, Best Fitness: 0
Generation 15, Best Fitness: 0
Generation 16, Best Fitness: 0
Generation 17, Best Fitness: 0
Generation 18, Best Fitness: 0
Generation 19, Best Fitness: 0
Generation 20, Best Fitness: 0
Best sequence: tensor([71, 69, 66, 66, 62, 60, 63, 63])
